<a href="https://colab.research.google.com/github/ingabLee/Transformers_book/blob/main/TransformerLearning_Chap43_49.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# load model, tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# prepare dictionary for label
dic = { 0:'positive', 1:'neutral', 2:'negative' }

# data
eval_list = ["I like apple",
             "I like pear",
             "I go to school",
             "I dislike mosquito",
             "I felt very sad",
             "I feel so good"]

# answer label
ans = torch.tensor([0, 0, 1, 2, 2, 0])

# no fine-tunning inference
# eval mode
model.eval()

# no graident
# eval mode is no train.
with torch.no_grad():
  for article in eval_list:

    # eval_list text -> encoding tokenizing
    inputs = tokenizer.encode(article, return_tensors="pt", padding=True, truncation=True)

    # inputs model
    outputs = model(inputs)

    # get logits
    logits = outputs.logits

    # argmax output logits
    # convert using item()
    # print label with input text
    print(f"{dic[logits.argmax(-1).item()]}:{article}")


negative:I like apple
negative:I like pear
negative:I go to school
negative:I dislike mosquito
positive:I felt very sad
positive:I feel so good


In [ ]:
# runtime 1min
# activation func AdamW
from torch.optim import AdamW

# instance AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

# tran mode.
model.train()

# epoch 50, loss list
epochs=50
losses = []

# fine-tunning
for epoch in range(epochs):
  # init gradient
  optimizer.zero_grad()

